In [36]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from random import uniform
import pandas as pd
import sqlite3 
import xarray as xr

%matplotlib inline
%config InlineBackend.figure_format='retina'

# 1) 50 most traveled airports

In [37]:
df_code = pd.read_csv('hw_5_data/ICAO_airports.csv')
df_top = pd.read_csv('hw_5_data/top_airports.csv')

In [38]:
df = pd.merge(df_top, df_code, how='left', left_on=['ICAO'], right_on=['ident'])[['Airport', 'wikipedia_link', 'latitude_deg', 'longitude_deg']]

In [39]:
df = df.astype(object).replace(np.nan, 'None')

In [40]:
df.head()

,Airport,wikipedia_link,latitude_deg,longitude_deg
0,Hartsfield-Jackson Atlanta International Airport,http://en.wikipedia.org/wiki/Hartsfield-Jackso...,33.636700,-84.428101
1,Chicago O'Hare International Airport,http://en.wikipedia.org/wiki/O'Hare_Internatio...,41.978600,-87.904800
2,Los Angeles International Airport,http://en.wikipedia.org/wiki/Los_Angeles_Inter...,33.942501,-118.407997
3,Dallas/Fort Worth International Airport,http://en.wikipedia.org/wiki/Dallas-Fort_Worth...,32.896801,-97.038002
4,Denver International Airport,http://en.wikipedia.org/wiki/Denver_Internatio...,39.861698,-104.672997


In [49]:
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()
sql_cmd = """CREATE TABLE my_table (
    airport TEXT,
    wikipedia TEXT, 
    latitude FLOAT,
    longitude FLOAT)
    """
cursor.execute(sql_cmd)

for i, row in df.iterrows():
    # print(i, row)
    _airport = row['Airport']
    _wikipedia = row['wikipedia_link']
    _latitude = row['latitude_deg']
    _longitude = row['longitude_deg']
    data = (_airport, _wikipedia, _latitude, _longitude)
    # print(data)
    # sql_cmd = f"INSERT INTO my_table (airport, wikipedia, latitude, longitude) VALUES ({_airport}, {_wikipedia}, {_latitude}, {_longitude})"
    sql_cmd = ("INSERT INTO my_table (airport, wikipedia, latitude, longitude) VALUES " + str(data))
    cursor.execute(sql_cmd)
    connection.commit()


# 2) Weather data

In [50]:
# temp: tasmax/tasmin, RH: rhsmax/rhsmin, precipitation: pr
# downward shortwave solar radiation(rsds), eastward wind(uas), 
# northward wind(vas), and specific humidity(huss)

def read_data(name):

    data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{name}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_daily.nc"
            )


    data_tmp = xr.open_dataset(data_path)
    return data_tmp.loc[dict(time=slice("1990-01-01", "2000-01-01"))]

max_temp_xr  = read_data('tasmax')
min_temp_xr = read_data('tasmin')
max_hum_xr = read_data('rhsmax')
min_hum_xr = read_data('rhsmin')
pr_xr = read_data('pr')

In [51]:
max_temp_xr['air_temperature']

<xarray.DataArray 'air_temperature' (time: 3653, lat: 585, lon: 1386)>
[2961888930 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) datetime64[ns] 1990-01-01 1990-01-02 ... 2000-01-01
Attributes:
    long_name:      Daily Maximum Near-Surface Air Temperature
    units:          K
    grid_mapping:   crs
    standard_name:  air_temperature
    height:         2 m
    cell_methods:   time: maximum(interval: 24 hours)
    _ChunkSizes:    [162  51 123]

In [52]:
max_temp_xr.sel(lat=39, lon=250.6996765137, method='nearest')['air_temperature']

<xarray.DataArray 'air_temperature' (time: 3653)>
array([276.77737, 276.44754, 274.65933, ..., 270.29263, 269.3296 , 271.8547 ],
      dtype=float32)
Coordinates:
    lat      float64 38.98
    lon      float64 250.7
  * time     (time) datetime64[ns] 1990-01-01 1990-01-02 ... 2000-01-01
Attributes:
    long_name:      Daily Maximum Near-Surface Air Temperature
    units:          K
    grid_mapping:   crs
    standard_name:  air_temperature
    height:         2 m
    cell_methods:   time: maximum(interval: 24 hours)
    _ChunkSizes:    [162  51 123]

In [ ]:
for lat, lon in zip(df['latitude_deg'], df['latitude_deg']):
    lon += 360
    print(max_temp_xr.sel(lat=lat, lon=lon, method='nearest')['air_temperature'])

In [ ]:
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()
sql_cmd = """CREATE TABLE my_table (
    lat FLOAT,
    lon FLOAT, 
    time INT,
    max_temp FLOAT,
    min_temp FLOAT,
    max_hum FLOAT,
    min_hum FLOAT,
    pr FLAOT)
    """
cursor.execute(sql_cmd)